<a href="https://colab.research.google.com/github/sogangori/choongang20/blob/master/keras_logistic_regression_expert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

공유 : http://bitly.kr/vAZP7yRR  , github.com/sogangori/choongang20


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.datasets import load_iris

In [28]:
x, y = load_iris(return_X_y=True)
x.shape, y.shape, set(y) # 150개의 데이터, 클래스 k=3

((150, 4), (150,), {0, 1, 2})

In [0]:
class MyModel(keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()#상속한 클래스의 생성자 호출 
    self.opt = tf.optimizers.SGD(learning_rate=0.01)#Stochatic Gradient Descent 확률적 경사 하강
    self.dense = keras.layers.Dense(units=3, activation=keras.activations.softmax)
  
  def call(self, x):
    h = self.dense(x)    
    return h

  def get_loss(self, y, h):
    cross_entropy = - (y * tf.math.log(h) + (1 - y) * tf.math.log(1 - h)) 
    loss = tf.reduce_mean(cross_entropy)
    return loss

  def get_accuracy(self, y, h):
    # h 3개의 확률: (m, 3), y: (m)
    predict = tf.argmax(h, -1)
    self.acc = tf.reduce_mean(tf.cast(tf.equal(y, predict), tf.float32)) # True > 1, False > 0 로 cast

  def fit(self, x, y, epoch=1):
    # x : (m, 4), y: (m)    
    y_hot = tf.one_hot(y, depth=3, axis=-1)#(m, 3)    
    for i in range(epoch):
      with tf.GradientTape() as tape: #경사 기록 장치
        h = self.call(x)
        loss = self.get_loss(y_hot, h)
      grads = tape.gradient(loss, self.trainable_variables) #경사 계산
      self.opt.apply_gradients(zip(grads, self.trainable_variables)) # 가중치에서 경사를 빼기
      self.get_accuracy(y, h)
      print('%d/%d loss:%.3f acc:%.3f'%(i, epoch, loss, self.acc))
model = MyModel()

In [64]:
model.fit(x, y, 10)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

0/10 loss:1.033 acc:0.647
1/10 loss:1.018 acc:0.653
2/10 loss:1.002 acc:0.653
3/10 loss:0.987 acc:0.653
4/10 loss:0.972 acc:0.653
5/10 loss:0.957 acc:0.653
6/10 loss:0.942 acc:0.653
7/10 loss:0.927 acc:0.653
8/10 loss:0.912 acc:0.653
9/10 loss:0.898 acc:0.653


In [60]:
h = model(x[:1])
print(np.array(h)) #확률

[[0.7783411  0.18895338 0.03270547]]
